In [ ]:
!wget "http://skuld.cs.umass.edu/traces/mmsys/2014/user06.tar"
!tar -xvf user06.tar

--2025-11-18 21:14:50--  http://skuld.cs.umass.edu/traces/mmsys/2014/user06.tar
Resolving skuld.cs.umass.edu (skuld.cs.umass.edu)... 128.119.245.5
Connecting to skuld.cs.umass.edu (skuld.cs.umass.edu)|128.119.245.5|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5307791360 (4.9G) [application/x-tar]
Saving to: ‘user06.tar’

user06.tar          100%[===================>]   4.94G  22.6MB/s    in 2m 54s  

2025-11-18 21:17:44 (29.1 MB/s) - ‘user06.tar’ saved [5307791360/5307791360]

user06/
user06/YawDD dataset.rar


In [ ]:
!ls
!ls user06

sample_data  user06  user06.tar
'YawDD dataset.rar'


In [ ]:
!ls -R user06

user06:
'YawDD dataset.rar'


In [ ]:
!apt-get install unrar
!unrar x "user06/YawDD dataset.rar" dataset/

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from user06/YawDD dataset.rar

Creating    dataset                                                   OK
Creating    dataset/YawDD dataset                                     OK
Creating    dataset/YawDD dataset/Dash                                OK
Creating    dataset/YawDD dataset/Dash/Female                         OK
Extracting  dataset/YawDD dataset/Dash/Female/1-FemaleNoGlasses.avi        0%  1%  2%  3%  OK 
Extracting  dataset/YawDD dataset/Dash/Female/10-FemaleNoGlasses.avi       3%  4%  OK 
Extracting  dataset/YawDD dataset/Dash/Female/11-FemaleGlasses.avi.avi       4%  5%  6%  OK 
Extracting  dataset/YawDD dataset/Dash/Female

In [ ]:
!ls dataset

'YawDD dataset'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p "/content/drive/MyDrive/YawDD"

In [ ]:
!cp -r "/content/dataset/YawDD" "/content/drive/MyDrive/YawDD/"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
data_path = "/content/drive/MyDrive/YawDD/YawDD"
for root, dirs, files in os.walk(data_path):
    level = root.replace(data_path, "").count(os.sep)
    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * 4 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")

YawDD/
    Readme_YawDD.pdf
    Table1.pdf
    Table2.pdf
    Dash/
        Female/
            1-FemaleNoGlasses.avi
            10-FemaleNoGlasses.avi
            11-FemaleGlasses.avi.avi
            12-FemaleGlasses.avi.avi
            13-FemaleGlasses.avi.avi
            2-FemaleNoGlasses.avi
            3-FemaleGlasses.avi
            4-FemaleNoGlasses.avi
            5-FemaleNoGlasses.avi
            6-FemaleNoGlasses.avi
            7-FemaleNoGlasses.avi
            8-FemaleGlasses.avi.avi
            9-FemaleNoGlasses.avi
        Male/
            1-MaleGlasses.avi
            10-MaleGlasses.avi
            11-MaleGlasses.avi
            12-MaleGlasses.avi
            13-MaleNoGlasses .avi
            14-MaleNoGlasses.avi
            15-MaleGlasses.avi
            16-MaleNoGlasses.avi
            2-MaleGlasses.avi
            3-MaleGlasses.avi
            4-MaleNoGlasses.avi
            5-MaleGlasses.avi
            6-MaleGlasses.avi
            7-MaleGlasses.avi
            8-

In [ ]:
!apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
!pip install opencv-python imutils dlib tqdm pandas huggingface_hub -q

In [ ]:
import os
if not os.path.exists('/content/shape_predictor_68_face_landmarks.dat'):
    !wget -q http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
    !bunzip2 -q shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import cv2
import dlib
import numpy as np
import pandas as pd
from imutils import face_utils
from scipy.spatial import distance as dist
from glob import glob
from tqdm import tqdm
import os

In [ ]:
RAW_YAWDD = "/content/drive/MyDrive/ML-Project-Dataset/YawDD"
PROCESSED_ROOT = "/content/processed/YawDD"
os.makedirs(f"{PROCESSED_ROOT}/faces/alert", exist_ok=True)
os.makedirs(f"{PROCESSED_ROOT}/faces/drowsy", exist_ok=True)
os.makedirs(f"{PROCESSED_ROOT}/eyes/alert", exist_ok=True)
os.makedirs(f"{PROCESSED_ROOT}/eyes/drowsy", exist_ok=True)
os.makedirs(f"{PROCESSED_ROOT}/mouth/alert", exist_ok=True)
os.makedirs(f"{PROCESSED_ROOT}/mouth/drowsy", exist_ok=True)

In [ ]:
# loading dlib models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks.dat")

In [ ]:
# Feature functions
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[14], mouth[18])
    C = dist.euclidean(mouth[12], mouth[16])
    return A / C

def circularity(eye):
    A = dist.euclidean(eye[1], eye[4])
    radius = A / 2.0
    Area = np.pi * (radius ** 2)
    perimeter = sum(dist.euclidean(eye[i], eye[(i+1)%6]) for i in range(6))
    return 4 * np.pi * Area / (perimeter**2) if perimeter > 0 else 0

def mouth_over_eye(left_eye, right_eye, mouth):
    ear = (eye_aspect_ratio(left_eye) + eye_aspect_ratio(right_eye)) / 2.0
    mar = mouth_aspect_ratio(mouth)
    return mar / ear if ear > 0 else 0

In [ ]:
# no of vids
video_paths = glob(f"{RAW_YAWDD}/**/*.avi", recursive=True) + glob(f"{RAW_YAWDD}/**/*.mov", recursive=True)
print(f"Found {len(video_paths)} videos")

Found 349 videos


In [ ]:
# Storage
all_features = []
total_saved = 0

for video_path in tqdm(video_paths):
    video_name = os.path.basename(video_path)
    label = "drowsy" if any(k in video_name.lower() for k in ["sleepy", "yawning", "slow blink", "tired"]) else "alert"

    cap = cv2.VideoCapture(video_path)
    frame_idx = 0
    saved_this_video = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1
        # sample every 5th frame
        if frame_idx % 5 != 0:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray) # detect face

        if len(faces) == 0:
            continue

        face = faces[0]
        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)

        # Regions
        (x, y, w, h) = face_utils.rect_to_bb(face)
        face_crop = frame[y:y+h, x:x+w]
        if face_crop.size == 0:
            continue

        left_eye_pts = shape[36:42]
        right_eye_pts = shape[42:48]
        mouth_pts = shape[48:68]

        # Bounding boxes
        left_box = cv2.boundingRect(left_eye_pts)
        right_box = cv2.boundingRect(right_eye_pts)
        mouth_box = cv2.boundingRect(mouth_pts)

        (lx, ly, lw, lh) = left_box
        (rx, ry, rw, rh) = right_box
        (mx, my, mw, mh) = mouth_box

        left_eye_crop = frame[ly:ly+lh, lx:lx+lw]
        right_eye_crop = frame[ry:ry+rh, rx:rx+rw]
        mouth_crop = frame[my:my+mh, mx:mx+mw]

        # Resize
        try:
            face_crop = cv2.resize(face_crop, (224, 224))
            left_eye_crop = cv2.resize(left_eye_crop, (64, 64))
            right_eye_crop = cv2.resize(right_eye_crop, (64, 64))
            mouth_crop = cv2.resize(mouth_crop, (128, 64))
        except:
            continue

        # Save
        base = f"{os.path.splitext(video_name)[0]}_f{frame_idx}"
        cv2.imwrite(f"{PROCESSED_ROOT}/faces/{label}/{base}.jpg", face_crop)
        cv2.imwrite(f"{PROCESSED_ROOT}/eyes/{label}/{base}_left.jpg", left_eye_crop)
        cv2.imwrite(f"{PROCESSED_ROOT}/eyes/{label}/{base}_right.jpg", right_eye_crop)
        cv2.imwrite(f"{PROCESSED_ROOT}/mouth/{label}/{base}.jpg", mouth_crop)

        # Features
        ear = (eye_aspect_ratio(left_eye_pts) + eye_aspect_ratio(right_eye_pts)) / 2.0
        mar = mouth_aspect_ratio(mouth_pts)
        cir = (circularity(left_eye_pts) + circularity(right_eye_pts)) / 2.0
        moe = mouth_over_eye(left_eye_pts, right_eye_pts, mouth_pts)

        all_features.append({
            'video': video_name,
            'frame': frame_idx,
            'label': 1 if label == 'drowsy' else 0,
            'ear': round(ear, 4),
            'mar': round(mar, 4),
            'circularity': round(cir, 4),
            'moe': round(moe, 4)
        })

        saved_this_video += 1
        total_saved += 1

        if saved_this_video >= 200:  # max 200 frames per video
            break

    cap.release()

100%|██████████| 349/349 [57:49<00:00,  9.94s/it]


In [ ]:
# Save features CSV
df = pd.DataFrame(all_features)
df.to_csv(f"{PROCESSED_ROOT}/geometric_features.csv", index=False)
print(f"\nDONE! Saved {total_saved} frames + features from {len(video_paths)} videos")
print(f"Features shape: {df.shape}")


DONE! Saved 36524 frames + features from 349 videos
Features shape: (36524, 7)


In [ ]:
df.to_csv("geometric_features_yawdd.csv", index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # ← run this first, click the link, allow access

In [ ]:
# Zip and save directly to your Drive in one command
!zip -r "/content/drive/MyDrive/YawDD_dataset.zip" "/content/processed/YawDD"

print("Done! Your zip is now safely saved in My Drive → YawDD_dataset.zip")